In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date

# Input Variables
expiry_date = "2022-08-19"
product_type = "ETF"  # ETF / Equity/ leave blank

# Web-scrape tradeable option products from cboe.com
def scrape_weekly_option_products():
    url = "https://www.cboe.com/available_weeklys/"
    weeklys = pd.read_html(url)[1]
    return weeklys

# Read option products from pre-formatted file (source: cboe.com)
def read_weekly_option_products(product_type=None):
    weekly_products = pd.read_csv("weeklysmf.csv")

    if product_type == "ETF":
        weekly_products = weekly_products[weekly_products.Type == "ETF"]

    if product_type == "Equity":
        weekly_products = weekly_products[weekly_products.Type == "Equity"]

    return weekly_products.iloc[:, 0]

# Scrape option chain for the symbol and extract data for at-the-money (atm) calls and puts
def get_atm_option_data(symbol):
    ticker = yf.Ticker(symbol)
    last_price = ticker.history(period=expiry_date).Close[0]
    calls = ticker.option_chain(date=expiry_date).calls
    puts = ticker.option_chain(date=expiry_date).puts
    atm_call = calls[calls.inTheMoney == True].iloc[-1, :]
    atm_put = puts[puts.inTheMoney == False].iloc[-1, :]
    strike = atm_put.strike
    atm_call_last = (atm_call.bid + atm_call.ask) / 2
    atm_put_last = (atm_put.bid + atm_put.ask) / 2
    atm_call_IV = atm_call.impliedVolatility
    atm_put_IV = atm_put.impliedVolatility
    atm_data = [
        symbol,
        last_price,
        strike,
        atm_call_last,
        atm_put_last,
        atm_call_IV,
        atm_put_IV,
    ]
    return atm_data

# Execution loop that writes the atm option data for each symbol into a single csv file. 
def execute():
    output_array = []
    option_products = read_weekly_option_products(product_type)
    for product in option_products:
        try:
            product_data = get_atm_option_data(product)
            output_array.append(product_data)
        except:
            print("pass")
    option_data_df = pd.DataFrame(
        data=output_array,
        columns=[
            "symbol",
            "last_price",
            "strike",
            "atm_call_last",
            "atm_put_last",
            "atm_call_IV",
            "atm_put_IV",
        ],
    )
    option_data_df.to_csv("option_data_" + expiry_date + "_" + str(date.today()) + ".csv")


if __name__ == "__main__":
    execute()